FE

In [ ]:
import pandas as pd

In [ ]:
# 오류(SettingWithCopyError 발생)
pd.set_option('mode.chained_assignment', 'raise') # SettingWithCopyError

# 경고(SettingWithCopyWarning 발생, 기본 값입니다)
pd.set_option('mode.chained_assignment', 'warn') # SettingWithCopyWarning

# 무시
pd.set_option('mode.chained_assignment',  None) # <==== 경고를 끈다

In [ ]:
is_Train = True
import numpy as np
from tqdm import tqdm
dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
}
if is_Train:
    df = pd.read_csv('input/data/train_data.csv', dtype=dtype, parse_dates=['Timestamp'])
else:
    df = pd.read_csv('input/data/test_data.csv', dtype=dtype, parse_dates=['Timestamp'])
df = df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
df['endtime'] = np.roll(df['Timestamp'], -1)
df['totaltime'] = (df['endtime'] - df['Timestamp'])
time_ = list()
for i in df['totaltime']:
    a = i.total_seconds()
    if a > 3600:
        a = 3600
    elif a < 0:
        a = -1
    time_.append(a)
time_[-1] = -1

def grouping(a):
    if a < 0:
        return -1
    if a == 0:
        return 0
    elif 0 < a <= 10:
        return 1
    elif 10 < a <= 20:
        return 2
    elif 20 < a <= 40:
        return 3
    elif 40 < a <= 60:
        return 4
    elif 60 < a <= 120:
        return 5
    elif 120 < a <= 600:
        return a//60 + 3
    else: 
        return a//600 + 13
# (-1~20) feat
df['totaltime'] = [grouping(i) + 1 for i in time_]
if is_Train:
    np.save('/opt/ml/train_col/totaltime.npy', df['totaltime'])
else:
    np.save('/opt/ml/test_col/totaltime.npy', df['totaltime'])


In [ ]:
if is_Train:
    for i in [2,4,5,6,8,9]:
        np.save('/opt/ml/train_col/assessmentItemID_' + str(i) + '.npy', np.array([j[i] for j in df['assessmentItemID']]))
else:
    for i in [2,4,5,6,8,9]:
        np.save('/opt/ml/test_col/assessmentItemID_' + str(i) + '.npy', np.array([j[i] for j in df['assessmentItemID']]))

In [ ]:
from tqdm import tqdm
li = list()
for i in tqdm(np.unique(df['userID'])):
    a = df[df['userID'] == i]
    li += acc(a).to_list()
if is_Train:
    np.save('/opt/ml/train_col/acc_list.npy', np.array(li))
else:
    np.save('/opt/ml/test_col/acc_list.npy', np.array(li))
    

In [ ]:
df_train = pd.read_csv('input/data/train_data.csv', dtype=dtype, parse_dates=['Timestamp'])
df_test = pd.read_csv('input/data/test_data.csv', dtype=dtype, parse_dates=['Timestamp'])
df_train = df_train.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)
df_test = df_test.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

In [ ]:
set(df_train['assessmentItemID'].tolist()) == set(df_test['assessmentItemID'].tolist())

In [ ]:
assess_corr = {}
from tqdm import tqdm
for i in tqdm(set(df_train['assessmentItemID'].tolist())):
    tmp = df_train[df_train.assessmentItemID == i]
    if sum(tmp.answerCode)/tmp.shape[0]>0.5:
        assess_corr[i] = 1
    else:
        assess_corr[i] = 0
    
if is_Train:
    np.save('/opt/ml/train_col/item_correct.npy',[assess_corr[i] for i in df['assessmentItemID']])
else:
    np.save('/opt/ml/test_col/item_correct.npy',[assess_corr[i] for i in df['assessmentItemID']])


100%|██████████| 9454/9454 [24:20<00:00,  6.47it/s]


In [ ]:
item_corr = {}
for i in tqdm(set(df_train['testId'].tolist())):
    tmp = df_train[df_train.testId == i]
    if sum(tmp.answerCode)/tmp.shape[0]>0.5:
        item_corr[i] = 1
    else:
        item_corr[i] = 0

if is_Train:
    np.save('/opt/ml/train_col/test_correct.npy',[item_corr[i] for i in df['testId']])
else:
    np.save('/opt/ml/test_col/test_correct.npy',[item_corr[i] for i in df['testId']])


100%|██████████| 1537/1537 [03:59<00:00,  6.40it/s]
